In [1]:
import numpy as np
import pandas as pd
import sklearn 
import matplotlib.pyplot as plt

from pylab import figure
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.mplot3d.art3d import Line3DCollection
np.set_printoptions(suppress=True)

<h2>PRACTICA 3.  APARTADO 1.</h2> 

Para realizar la estandarización de datos, prepararemos el data frame para que esta sea adecuada. Transformaremos la columna Ciudad, categorizandola mediante numeros enteros, despues usaremos el método StandardScaler para estandarizar los datos.

In [2]:

import keras_metrics



from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras import optimizers
from tensorflow.keras import utils as k
from tensorflow.keras.callbacks import EarlyStopping



file= 'infarto.csv'
dt=pd.read_csv(file, sep=";")


#Aunque no seamos expertos en el tema entendemos que las columnas casado y trabajo no aportan nada para el modelo
#La residencia si porque la calidad del aire es diferente

dt=dt.drop(['id','casado', 'trabajo'], axis=1)

print("Numero de filas con NaN values:\n",dt.isnull().sum())
print("\n\nComo no hay muchos NaN las filas que cuenten con valores nulos las descartamos")



from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

#Transformamos las columnas necesarias con to_categorical
#antes 
columna1=np.array(dt["genero"])
columna2=np.array(dt["residencia"])
columna3=np.array(dt["uso_tabaco"])
label_encoder = LabelEncoder()

genero = label_encoder.fit_transform(columna1)
residencia= label_encoder.fit_transform(columna2)
tabaco= label_encoder.fit_transform(columna3)


dt["genero"]= k.to_categorical(genero)
dt["residencia"]= k.to_categorical(residencia)
dt["uso_tabaco"]= k.to_categorical(tabaco)

def minmax_norm(datos_input): 
    return (datos_input - dt.min()) / ( dt.max() - dt.min())
dt=minmax_norm(dt)


dt=dt.drop(dt.index[249:len(dt)])
dt=dt.dropna()
dt

Numero de filas con NaN values:
 genero                    0
edad                      0
hipertensión              0
enfermedad_coronaria      0
residencia                0
nivel_glucosa             0
bmi                     201
uso_tabaco                0
infarto                   0
dtype: int64


Como no hay muchos NaN las filas que cuenten con valores nulos las descartamos


,genero,edad,hipertensión,enfermedad_coronaria,residencia,nivel_glucosa,bmi,uso_tabaco,infarto
0,0.0,0.816895,0.0,1.0,0.0,0.801265,0.301260,1.0,1.0
2,0.0,0.975586,0.0,1.0,1.0,0.234512,0.254296,0.0,1.0
3,1.0,0.597168,0.0,0.0,0.0,0.536008,0.276060,0.0,1.0
4,1.0,0.963379,1.0,0.0,1.0,0.549349,0.156930,0.0,1.0
5,0.0,0.987793,0.0,0.0,0.0,0.605161,0.214204,1.0,1.0
...,...,...,...,...,...,...,...,...,...
243,1.0,0.829102,1.0,1.0,0.0,0.888145,0.345934,1.0,1.0
244,0.0,0.694824,0.0,0.0,1.0,0.137753,0.302405,0.0,1.0
245,1.0,0.169922,0.0,0.0,1.0,0.012972,0.235968,0.0,1.0
246,1.0,0.914551,0.0,0.0,1.0,0.109316,0.217640,1.0,1.0



<h2>PRACTICA 3.  APARTADO 2.</h2> 

Obtendremos la matriz de covarianza y calcularemos los autovectores y autovalores. Con estos ultimos datos podremos obtener los componentes y su porcentaje acumulado. Con los resultados comprobamos que con 3 componentes seremos capaces de mostrar el 70% de información, si usaramos solo dos componentes perderiamos casi el 50% de la información.

In [3]:
m_cov = np.cov(dt.T)
#Obtenemos los autovectores y los autovalores
eig_vals, eig_vecs = np.linalg.eig(m_cov) 

print('Autovectores\n%s' %eig_vecs)
print('\nAutovalores\n%s' %eig_vals)

Autovectores
[[ 0.87523742 -0.07669006  0.38865891  0.26628832 -0.02431271 -0.07430378
  -0.00084259 -0.00005778  0.        ]
 [-0.02641463  0.00836224  0.0644106  -0.02003762 -0.04295512 -0.02940069
  -0.97532657 -0.2017688   0.        ]
 [ 0.12186274 -0.04052144 -0.11772698 -0.29182738 -0.93221727  0.12030311
   0.03084558  0.00559475  0.        ]
 [-0.30639769  0.04607989  0.88219939 -0.19231956 -0.05426185  0.28561449
   0.06491029 -0.00108959  0.        ]
 [-0.0848258  -0.99388198  0.02367151 -0.04568484  0.03988373 -0.02629358
  -0.00302419 -0.00803352  0.        ]
 [-0.14627164  0.04097172  0.21908291 -0.10255923 -0.13637271 -0.94269413
   0.03468891  0.0996853   0.        ]
 [-0.01039146  0.01015496  0.00772705 -0.00000169 -0.01338687 -0.08966729
   0.20572022 -0.97426328  0.        ]
 [-0.30956883 -0.02646071  0.06358259  0.89102946 -0.32438084  0.01273891
   0.00626928  0.00813729  0.        ]
 [ 0.          0.          0.          0.          0.          0.
   0.          0.

In [4]:
print('Porcentaje por cada componente: \n')
suma = sum(eig_vals) 
num2 = eig_vals * 100 / suma
num2

Porcentaje por cada componente: 



array([23.21395874, 21.48491819, 12.75119956, 16.22619129, 17.92197464,
        6.14749934,  1.92736074,  0.32689748,  0.        ])

In [5]:
print('Porcentaje acumulado por cada componente: \n')
num2 = num2.cumsum()
num2.real

Porcentaje acumulado por cada componente: 



array([ 23.21395874,  44.69887694,  57.4500765 ,  73.67626779,
        91.59824244,  97.74574177,  99.67310252, 100.        ,
       100.        ])

<h2>PRACTICA 3.  APARTADO 3.</h2> 

Obtenemos los tres componentes principales y graficamos los datos en un diagrama xyz que los muestra como estan correlacionados los datos entre si.

In [8]:
#Calculamos tres Componentes con los autovectores
Componente1 = eig_vecs[0]
Componente2 = eig_vecs[1]
Componente3 = eig_vecs[2]
Componente4 = eig_vecs[3]
Componente5 = eig_vecs[4]

#Mostramos los Componentes
dfComp = pd.DataFrame()
dfComp['C1'] = Componente1
dfComp['C2'] = Componente2
dfComp['C3'] = Componente3
dfComp['C4'] = Componente4
dfComp['C5'] = Componente5

#dfD = pd.DataFrame(data = dt.dot(dfComp))
#dfD


dfComp
#Componente1,Componente2,Componente3


,C1,C2,C3,C4,C5
0,0.875237,-0.026415,0.121863,-0.306398,-0.084826
1,-0.076690,0.008362,-0.040521,0.046080,-0.993882
2,0.388659,0.064411,-0.117727,0.882199,0.023672
3,0.266288,-0.020038,-0.291827,-0.192320,-0.045685
4,-0.024313,-0.042955,-0.932217,-0.054262,0.039884
5,-0.074304,-0.029401,0.120303,0.285614,-0.026294
6,-0.000843,-0.975327,0.030846,0.064910,-0.003024
7,-0.000058,-0.201769,0.005595,-0.001090,-0.008034
8,0.000000,0.000000,0.000000,0.000000,0.000000


<h2>PRACTICA 3.  APARTADO 4.</h2> 

Obtenemos los tres componentes principales y graficamos los datos con la libreria scikit-learn.